In [1]:
import pandas as pd
import os
import math

In [2]:
"""Functions Related to Tranfering Data to Excel Sheet"""
def writeDataframe6(wbpath,df,sheetname,dfcols,xlcols,indexcol):    
    wb=opxl.load_workbook(wbpath)
    wsheet=wb[sheetname]    
    for index,row in df.iterrows():        
        myrow=int(row[indexcol])
        for idf,ixl in zip(dfcols,xlcols):
            wsheet.cell(row=myrow,column=ixl).value=row[idf]               
    wb.save(wbpath)
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()  

In [3]:
class typeAreach:
    def __init__(self,reach_name,fillHeight,Length):
        self.Name=reach_name
        self.h=fillHeight
        self.length=Length
        self.n=3
        self.b=4.3        
        a=(self.b+self.n*self.h)*self.h
        self.area=round(a,3)
        p=(self.b+2*self.h*math.sqrt(math.pow(self.n,2)+1))
        self.perimeter=round(p,3)
    def calcualteUnitBoq(self):
        pass        
    def display(self):
        print("Name={} h={} L={} a={} p={}".format(self.Name,self.h,self.length,self.area,self.perimeter))
        
        
    
        

In [4]:
def calculateExtensionLength(n,t):
    ext=2*n*t+4*t*(math.sqrt(math.pow(n,2)+1)-n)
    return ext
def calculateBoQ(h,b,n,L,wtype):
    myboq=[0 for x in range(0,15)]
    h1=round(h-0.2-0.1-0.003-0.1,3)
    a=(b+n*h1)*h1
    a=round(a,3)
    p=(b+2*h1*math.sqrt(math.pow(n,2)+1))
    p=round(p,3)
    """adding earth work"""
    bprofile=math.ceil(L/30.0)
    total_ew=a*L
    ew_carried_earth=total_ew*0.5
    ew_mechanical_excavator=total_ew*0.5
    ew_royalty=total_ew*0.5
    myboq[0]=bprofile
    myboq[1]=ew_carried_earth
    myboq[2]=ew_mechanical_excavator
    myboq[3]=ew_royalty    
    if wtype=='A':
        """adding filters"""
        t=0.1
        t_half=0.1/2.0
        extension1=2*n*t_half+4*(t_half)*(math.sqrt(math.pow(n,2)+1)-n)
        extension2=2*n*t+4*t*(math.sqrt(math.pow(n,2)+1)-n)        
        print("extension={} extension2={}".format(extension1,extension2))
        sf_length=p+extension1
        sandfilter_volume=sf_length*0.1*L
        anchor_length=2.0        
        geotex_width=p+extension2+anchor_length
        geotex_area=geotex_width*L
        extension3=calculateExtensionLength(n,0.153)
        gf_length=p+extension3
        gf_filter_vol=gf_length*0.1*L
        gf1=gf_filter_vol*0.5
        gf2=gf_filter_vol*0.5
        myboq[4]=sandfilter_volume
        myboq[5]=geotex_area
        myboq[6]=gf1
        myboq[7]=gf2
        """adding cc block"""
        extension4=calculateExtensionLength(n,0.203)
        block_piticing_length=p+extension4
        block_piticing_length=round(block_piticing_length,2)
        bno=math.ceil(block_piticing_length/0.4)
        block_per_meter=math.ceil(bno*2.5)
        block1=block_per_meter*L
        print("pitching_length={} nos={} bloc/m={} total={}".format(block_piticing_length,
                                                           bno,block_per_meter,
                                                                   block1))
        block2=math.ceil(10*(L/0.3))
        block_volume=block1*(0.4*0.4*0.2)+block2*(0.3*0.3*0.3)
        vo11=round(block_volume*0.5,3)
        cc_in_joint=(2*0.3*0.2*0.5)*L
        myboq[8]=block1
        myboq[9]=block2
        myboq[10]=vo11
        myboq[11]=vo11
        myboq[13]=cc_in_joint
    if wtype=='B5':
        h2=round(h-0.3,3)
        b1=b+2*n*0.3
        p1=(b1+2*h2*math.sqrt(math.pow(n,2)+1))
        p2=(b+2*h*math.sqrt(math.pow(n,2)+1))
        
        bag_at_top=math.ceil(1*L+1)
        bag_toe=math.ceil(1.5*(L/0.675)*2)
        total_bag=bag_at_top+bag_toe
        myboq[5]=round(p1*L,3)
        myboq[12]=total_bag
        myboq[14]=round(p2*L,3)
        print("h1={} area={} perimeter={} protection_type={} ".format(h1,a,p,wtype))
    return myboq
        
def calCulateReachAveragedFillHeight(input_df):
    area=input_df.iloc[-1,6]
    width=(input_df.iloc[-1,0]-input_df.iloc[0,0])*1000
    h=round((area/width),2)
    return h
    
    
def addReachAveragedFillHeightData(input_df):
    for index,row in input_df.iterrows():
        if index>0:
            h_avg=(input_df.iloc[index,2]+input_df.iloc[index-1,2])/2.0
            delx=0
            delA=0
            delx=(input_df.iloc[index,0]-input_df.iloc[index-1,0])*1000
            delA=delx*h_avg
            input_df.iloc[index,3]=h_avg
            input_df.iloc[index,4]=delx
            input_df.iloc[index,5]=delA
        if index==1:
            input_df.iloc[index,6]=delA
        if index>1:
            input_df.iloc[index,6]=delA+input_df.iloc[index-1,6]
            
            
    return input_df
    
def calcualteFillHeight(Area,sideSlope,topWidth):
    A=Area
    n=sideSlope
    b=topWidth
    t1=math.pow(b,2)
    t2=4*A*n
    t3=t1+t2
    t4=math.sqrt(t3)
    t5=t4-b
    t6=2*n
    h3=t5/t6
    #print("t1={} t2={} t3={} t4={} t5={} t6={} h={}".format(t1,t2,t3,t4,t5,t6,h3))
    #h1=(-b+math.sqrt(math.pow(b,2)+4*A**n))/(2.0*n)
    h=round(h3,2)
    return h
def addFillHeighttoDf(input_df):
    for index,row in input_df.iterrows():
        area=row['Area']
        h=calcualteFillHeight(area,3,4.3)
        print(h)
        input_df.iloc[index,2]=h
    return input_df
def testDf(input_df):
    mydfs=[]
    for index,row in input_df.iterrows():
        sheetName=row['SheetName']
        mydf=pd.read_excel(input_path,sheet_name=sheetName)       
        mydf.fillna(0,inplace=True)
        mydf1=addFillHeighttoDf(mydf)
        mydf2=addReachAveragedFillHeightData(mydf1)
        print(mydf2)
        mydfs.append(mydf2)
    return mydfs
def addFillHeight_to_Reach_Df(reach_df):
    mydfs=[]
    for index,row in reach_df.iterrows():
        sheetName=row['SheetName']        
        mydf=pd.read_excel(input_path,sheet_name=sheetName)       
        mydf.fillna(0,inplace=True)
        mydf1=addFillHeighttoDf(mydf)
        mydf2=addReachAveragedFillHeightData(mydf1)
        h=calCulateReachAveragedFillHeight(mydf2)
        reach_df.iloc[index,4]=h
        #print(mydf2)
        mydfs.append(mydf2)
    myvalues={'reach_data':reach_df,'fill_height_cal':mydfs}
    return myvalues
def createBoQDf(boq_tempalte,reach_df):
    for index,row in reach_df.iterrows():
        name=row['Reach']
        h=row['fill_height']
        length=row['Length']
        work_type=row['type']
        myboq=calculateBoQ(h,4.3,3,length,work_type)
        boq_tempalte[name]=myboq
    print(boq_tempalte)
    return boq_tempalte
def createPackageBoq(boq_df_corrected):
    myboq=boq_df_corrected.iloc[:,0:4]
    return myboq
def findRechesForPacakge(reach_df,pack_name):
    myreaches=[]
    for index,row in reach_df.iterrows():
        reach=row['Reach']
        package=row['package']
        if package==pack_name:
            myreaches.append(reach)
    return myreaches  
def populatePacakgeBoQ(boq_df,reach_names,package_boq_df):
    n=4+len(reach_names)
    for reach in reach_names:
        #mylist=list(boq_df_corrected[reach])
        package_boq_df[reach]=boq_df[reach]
    package_boq_df['Total_quantity']=package_boq_df.iloc[:,4:n].sum(axis=1)
    package_boq_df['Amount']=package_boq_df['Rate']*package_boq_df['Total_quantity']
    
    #package_boq_df['Amount']=package_boq_df[]
    return package_boq_df

def assemblePacakgeDf(boq_df_corrected,package_name_df,reach_df):
    packges=[]
    for index,row in package_name_df.iterrows():
        pname=row['name']
        print("package={}".format(pname))
        myboq=createPackageBoq(boq_df_corrected)
        reaches=findRechesForPacakge(reach_df, pname)       
        print("reaches={}".format(reaches))
        if reaches:
            myboq=populatePacakgeBoQ(boq_df_corrected,reaches,myboq)        
        packges.append(myboq)
    return packges
        
    
        
        
    

In [5]:
myfolder=r'E:\Website_26_07_2020\cmis6\Civilworks cost\Slope Protection'
input_path=os.path.join(myfolder,'Input.xlsx')
output_path=os.path.join(myfolder,'Output.xlsx')
sheetName='Main'
reach_df=pd.read_excel(input_path,sheet_name=sheetName)
reach_df.fillna(0,inplace=True)
myvalues=addFillHeight_to_Reach_Df(reach_df)
reach_df_corrected=myvalues['reach_data']
mydfs=myvalues['fill_height_cal']
#mydfs=testDf(reach_df)
sheetName="BoQ"
boq_df=pd.read_excel(input_path,sheet_name=sheetName)
boq_df.fillna(0,inplace=True)
"""calcualting Boq"""
boq_df_corrected=createBoQDf(boq_df,reach_df_corrected)
"""Package_df"""
sheetName='Packages'
package_name_df=pd.read_excel(input_path,sheet_name=sheetName)
package_name_df.fillna(0,inplace=True)
package_name_df
packages=assemblePacakgeDf(boq_df_corrected,package_name_df,reach_df_corrected)


myframes=[]
mynames=[]

1.71
4.14
3.31
0.81
1.07
1.16
1.16
1.13
1.12
1.14
1.16
1.23
1.28
1.24
1.22
1.0
1.45
1.45
1.99
1.83
1.66
1.31
1.29
1.23
1.18
1.17
1.23
1.28
1.29
1.28
1.17
1.25
1.23
1.2
1.32
1.31
1.35
2.33
2.52
1.28
1.28
1.26
1.33
1.2
1.21
1.3
1.24
1.21
1.23
1.31
1.31
1.35
1.34
1.34
1.41
1.17
1.34
1.16
1.17
1.17
1.19
1.31
1.3
1.65
1.76
1.45
1.39
1.42
1.39
1.38
1.06
1.2
0.95
1.02
0.66
1.44
2.86
3.52
2.3
0.8
0.93
0.64
0.66
0.68
0.65
0.62
0.56
0.6
0.62
0.59
0.68
0.73
0.7
0.83
1.1
0.89
0.94
0.91
0.87
0.84
0.93
0.97
0.96
0.88
0.96
0.97
1.04
1.09
1.13
1.14
1.15
1.13
1.15
1.15
1.19
1.16
1.16
1.16
extension=0.33245553203367595 extension2=0.6649110640673519
pitching_length=17.71 nos=45 bloc/m=113 total=5763
extension=0.33245553203367595 extension2=0.6649110640673519
pitching_length=10.56 nos=27 bloc/m=68 total=34680
extension=0.33245553203367595 extension2=0.6649110640673519
pitching_length=11.7 nos=30 bloc/m=75 total=84000
h1=0.877 area=6.078 perimeter=9.847 protection_type=B5 
extension=0.33245553203367595 ext

In [6]:
myframes.append(reach_df_corrected)
mynames.append('Reach_input')
k=1
for df in mydfs:    
    print(df)
    h=calCulateReachAveragedFillHeight(df)    
    print("h={}".format(h))
    name="R"+str(k)
    length=(df.iloc[-1,0]-df.iloc[0,0])*1000
    length=round(length,2)
    myreach=typeAreach(name,h,length)
    myreach.display()
    myframes.append(df)
    mynames.append(name)    
    k +=1
myframes.append(boq_df_corrected)
mynames.append("BoQ")
for index,row in package_name_df.iterrows():
    p=row['name']
    p1=p.replace("/","_")
    p2=p1.replace("-","_")
    myframes.append(packages[index])
    mynames.append(p2)
    print(p2)
    


saveDataFrame(myframes,output_path,mynames)

   chainage   Area     h  h_avg  delx   delA  cumDelA
0    12.430  16.09  1.71  0.000   0.0   0.00     0.00
1    12.440  69.09  4.14  2.925  10.0  29.25    29.25
2    12.450  47.21  3.31  3.725  10.0  37.25    66.50
3    12.470   5.43  0.81  2.060  20.0  41.20   107.70
4    12.481   8.00  1.07  0.940  11.0  10.34   118.04
h=2.31
Name=R1 h=2.31 L=51.0 a=25.941 p=18.91
    chainage   Area     h  h_avg  delx   delA  cumDelA
0      19.59   9.00  1.16  0.000   0.0   0.00     0.00
1      19.60   8.97  1.16  1.160  10.0  11.60    11.60
2      19.65   8.70  1.13  1.145  50.0  57.25    68.85
3      19.70   8.60  1.12  1.125  50.0  56.25   125.10
4      19.75   8.77  1.14  1.130  50.0  56.50   181.60
5      19.80   8.97  1.16  1.150  50.0  57.50   239.10
6      19.85   9.77  1.23  1.195  50.0  59.75   298.85
7      19.90  10.37  1.28  1.255  50.0  62.75   361.60
8      19.95  10.00  1.24  1.260  50.0  63.00   424.60
9      20.00   9.75  1.22  1.230  50.0  61.50   486.10
10     20.05   7.31  1.00

In [7]:
boq_df

,ItemCode,Description,Unit,Rate,R1,R2,R3,R4,R5,R6,R7
0,16-100,Erection of bamboo profile,No,367.41,2.000000,17.000000,38.000000,24.000,25.000000,4.000000,60.000
1,16-220-10,Earth work by carried earth (by truck/boat or ...,Cum,429.88,487.305000,1313.760000,3843.280000,2127.300,2392.420000,705.250000,2575.810
2,16-240-10,Earth work by Mechanical Excavator (Long Boon)...,Cum,157.70,487.305000,1313.760000,3843.280000,2127.300,2392.420000,705.250000,2575.810
3,16-180,Royalty of specified earth taken from private ...,Cum,16.97,487.305000,1313.760000,3843.280000,2127.300,2392.420000,705.250000,2575.810
4,40-550-30,Supplying and laying sand as filter layers FM...,Cum,1267.96,85.136623,486.869232,1196.771020,0.000,772.001709,145.434555,0.000
5,40-500-40,Supplying and placing non-woven needle punched...,Sqm,250.13,970.321464,6058.244643,14580.060392,8608.645,9446.034187,1687.591106,17711.572
6,40-530-20,Supplying and laying shingles and peagravels a...,Cum,5771.61,44.314701,260.898505,636.737580,0.000,411.340615,76.141570,0.000
7,40-530-30,Supplying and laying shingles and peagravels a...,Cum,6135.23,44.314701,260.898505,636.737580,0.000,411.340615,76.141570,0.000
8,40-190-40,Manufacturing and supplying C.C. blocks in lea...,Nos,457.33,5763.000000,34680.000000,84000.000000,0.000,54020.000000,9800.000000,0.000
9,40-190-40,Manufacturing and supplying C.C. blocks in lea...,Nos,684.23,1700.000000,17000.000000,37334.000000,0.000,24667.000000,3334.000000,0.000


In [8]:
reach_df

,Reach,Start,Finish,Length,fill_height,type,package,SheetName,rowindex
0,R1,12.43,12.481,51,2.31,A,KISH/PW-30,R1,2
1,R2,19.59,20.100,510,1.18,A,KISH/PW-30,R2,3
2,R3,20.13,21.250,1120,1.36,A,KISH/PW-30,R3,4
3,R4,21.25,21.950,700,1.28,B5,KISH/PW-30,R4,5
4,R5,21.95,22.690,740,1.32,A,KISH/PW-31,R5,6
5,R6,26.08,26.180,100,1.97,A,KISH/PW-31,R6,7
6,R7,17.80,19.590,1790,0.90,B5,KISH/PW-31,R7,8


In [9]:
myboq=[0 for x in range(0,15)]
myboq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
ext=calculateExtensionLength(3,0.153)
extension4=calculateExtensionLength(3,0.203)
print(extension4)

1.349769460056724


In [11]:
package_name_df


,sl,name,short_name
0,1,KISH/PW-30,0.0
1,2,KISH/PW-31,0.0
2,3,KISH/PW-32,0.0
3,4,KISH/PW-33,0.0
4,5,KISH/PW-34,0.0
